In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from tqdm.notebook import tqdm
from result_saver import SaverProvider
from Scratch import metadata_loader
provider = SaverProvider()

In [3]:
df = metadata_loader(_extract=True)
df = df[~df['job_status'].isin(['JobStatus.DONE', 'JobStatus.ERROR', 'JobStatus.CANCELLED'])]

In [4]:
from time import sleep

start_idx = 0 # Initialize starting index

while True:  # Outer loop to restart when a specific error occurs
    try:
        for idx, job_id in enumerate(tqdm(df.job_id[start_idx:])):

            provider.retrieve_job(job_id, _ignore_running=False)

            start_idx = start_idx + 1  
        break  
    except KeyboardInterrupt:
        print('Interrupted')
        break  
    except Exception as e:
        if 'RequestsApiError' in str(type(e)):
            print(f'\nRequestsApiError occurred at job index {start_idx}, retrying...')
            sleep(5) 
        else:
            print(f'An unexpected error occurred: {e}')
            break  


  0%|          | 0/2 [00:00<?, ?it/s]

08:11:19 Warning: Job ID cne879d72scg008df4j0 not found in /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77
08:12:06 Warning: Job ID cne877wjpbeg008cst20 not found in /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77
